In [9]:
import pandas as pd
import csv
from emotion_predictor import EmotionPredictor
import re
import string

In [10]:
def strip_amp(text):
    text=text.replace("&amp;","")
    return text
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

In [11]:
def strip_mentions(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [12]:
# Pandas presentation options
pd.options.display.max_colwidth = 150   # show whole tweet's content
pd.options.display.width = 200          # don't break columns
# pd.options.display.max_columns = 7      # maximal number of columns
# df = pd.read_csv('sample_tweets.csv')
# Predictor for Ekman's emotions in multiclass setting.
tweets = []
flag = True
count = 0
c = 0
df = pd.read_csv('/Users/kavyasuresh/Downloads/JPMorgan.csv') 
for i,row in df.iterrows():
    count = count+1
    t = str(row.tweet)
    tweet = strip_links(strip_amp(t))
    tweets.append(strip_mentions(tweet))

In [ ]:

model = EmotionPredictor(classification='plutchik', setting='mc', use_unison_model=True)
# tweets = [
#     "Watching the sopranos again from start to finish!",
#     "Finding out i have to go to the  dentist tomorrow",
#     "Sun in my eyes but I don't mind, what a beautiful day we've had in New York today!",
#     "Feels like someone's stabbed me in my hope",
#     "Do people have no Respect for themselves or you know others peoples homes",
#     "I want to go outside and chalk but I have no chalk",
#     "I hate coming to the doctors when I feel as if I might have a big problem",
#     "My mom wasn't mad",
#     "You don't indicate once I'm already in the road THEN rev and honk at me you stupid bitch #learnhowtodrive #bitch",
#     "Come home from work and this is on my doorstep. I guess he has a secret admirer",
#     "The 'egyption hot models' facebook page is pathetic... simply photos of obese horny women.",
#     "I HATE PAPERS AH #AH #HATE",
# ]

# predictions = model.predict_classes(tweets)
# print(predictions, '\n')

probabilities = model.predict_probabilities(tweets)
# print(probabilities, '\n')

# embeddings = model.embedd(tweets)
# print(embeddings, '\n')
stress = []
angry = []
happy = []
satisfaction = []
for i,r in probabilities.iterrows():
    classes = {'Fear':r['Fear'],'Sadness':r['Sadness'],'Anticipation':r['Anticipation'],'Joy':r['Joy'],'Trust':r['Trust'],'Surprise':r['Surprise'],'Anger':r['Anger'],'Disgust':r['Disgust']}
    classes_sort = dict(sorted(classes.items(), key=lambda kv: kv[1]))
    rank = len(classes)
    for key, val in classes_sort.items():
        classes[key] = pow(rank * val,2)
        rank = rank - 1
    stress_val = classes['Fear']+classes['Sadness']+classes['Anticipation']-classes['Joy']-classes['Trust']-classes['Surprise']
    satisfaction_val = -classes['Fear']-classes['Anger']-classes['Disgust']+classes['Anticipation']+classes['Joy']+classes['Trust']
    happy_val = classes['Joy']+classes['Trust']+classes['Surprise']-classes['Sadness']-classes['Anger']-classes['Disgust']
    angry_val = classes['Anger']+classes['Anticipation']+classes['Disgust']-classes['Joy']-classes['Fear']-classes['Sadness']
    inpu = [stress_val,satisfaction_val,happy_val,angry_val]
    old_min = min(inpu)
    old_range = max(inpu) - old_min
    new_min = 0.00001
    new_range = 0.999999 - new_min
    output = [float((n - old_min) / old_range * new_range + new_min) for n in inpu]
    stress.append(output[0])
    satisfaction.append(output[1])
    happy.append(output[2])
    angry.append(output[3])
probabilities['Angry'] = angry 
probabilities['Happy'] = happy 
probabilities['Satisfaction'] = satisfaction 
probabilities['Stress'] = stress 


# predictions.to_csv("class.csv")
probabilities.to_csv("/Users/kavyasuresh/DS/Social-Stocking/Data set/jpmorgan_emo.csv")
#                      sep='\t')


inside init
models/unison-plutchik-mc.h5
